In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import random

In [61]:
def weight_init(experts,opening,method, tickers):
    #add market cap initialization
    if method == 'equal':
        w = np.ones(experts)
        w[opening.iloc[0]==0] = 0
        w[opening.iloc[0].isnull()] = 0
        return w.astype('float64')
    
    if method == 'marketcap':
        caps_dict = pd.read_csv('marketcapdata.csv', index_col='Name').to_dict()['Market Cap']
        w = []
        
        for ticker in tickers:
            try:
                w += caps_dict[ticker]
            except:
                w += 0
                
        return np.array(w).astype('float64')

def losses(opening,closing,method):
    if method == 'absolute': #absolute return (close-open)
        return ((closing-opening)<0).astype('float64')
    
    elif method=='pricerel': #price relative #closing over opening
        return ((closing-opening)<1).astype('float64') 
    
    else:
        print("Invalid method.")

In [62]:
def rwma(filename,lossmethod,weightmethod):
    #data is an N by T array with returns for each expert/stock i in time t forming a row
    #data = np.array(data)
    #data = totalassetdata
    data = pd.read_excel(filename)[::-1]
    opening = data.iloc[:,::2]
    closing = data.iloc[:,1::2]
    tickers = opening.columns
    dates = opening.index.values
    #print(dates)
    
    T = len(data)
    N = len(opening.columns)
    epsilon = np.sqrt(np.log(N)/T) 
   
    w = weight_init(N,opening,'equal', tickers)
    p = w/sum(w)
    
    start = 10000000
    
    #M = np.zeros(N)
    #M_wma_random = 0
    quantities = [] #$/prices = quantity of each
    #return quantities

    probs = [p]
    values =[]
    transaction = []
    #allweights = []

    # iterations
    for t in range(T):
               
        #find new prices for the day
        #opening_prices_na = opening.iloc[t]
        opening_prices = opening.iloc[t].values
        opening_prices[opening.iloc[t].isnull()] = 0
        if 0 not in opening_prices:
            w[w==0] = 1
        #closing_prices_na = closing.iloc[t]
        closing_prices = closing.iloc[t].values
        closing_prices[closing.iloc[t].isnull()] = 0
        #return opening_prices
        
        #portfolio value
        if t==0: 
            values.append(start)
        else:
            values.append(np.sum(np.array(opening_prices)*np.array(quantities[t-1]))) #new opening value
        
        q_t = []
        for j,k in enumerate(opening_prices):
            if k == 0:
                q_t.append(0)
            elif p[j] != 0:
                # TODO - fix true divide error
                q_t.append((p[j]*values[t])/k)
            else:
                q_t.append(0)
        quantities.append(q_t)

        loss = losses(opening_prices,closing_prices,lossmethod) 
        loss[np.isnan(loss)] = 0
                
        #check sum of weight updates
        #money_moved = sum(np.absolute([-p[i]*epsilon*loss[i] for i in range(N)]))*values[t]
        money_moved = sum(np.absolute([p[i]*((1-epsilon)**loss[i]-1) for i in range(N)]))*values[t]
        
        if t==0 or values[t]-values[t-1] >= 0.015*money_moved:
            for i in range(N):
                w[i] = w[i]*(1 - epsilon)**loss[i]
            transaction.append((t,0.015*money_moved))
        #print(w,transaction)
        
       
        p = w/sum(w)
        probs.append(p)
        #return p
    
    out = pd.DataFrame(probs[1:],columns=tickers,index=dates)
    #print(out)
    out['Values'] = values
    out.to_csv(f'rwma_weights_return_{weightmethod}_{lossmethod}_{filename[:-5]}.csv')
    print('Final distribution: ',p)
    print('Final transaction count: ',len(transaction),'total cost: ',sum([v[1] for v in transaction]))
    print('starting value: ',start,'final value: ',values[-1],'less transaction costs: ',values[-1]-sum([v[1] for v in transaction]))
    #np.savetxt('q_lv_w.csv',quantities,delimiter=',')
    print('Return',((values[-1]-sum([v[1] for v in transaction]))-start)/start*100,'%')
    plt.plot(values)
    plt.show()
    return values

In [63]:
filename = 'highvol_weekly.xlsx'

In [64]:
rwma(filename,'absolute','equal')

TypeError: ufunc 'true_divide' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''